#ORM
client                      server
python                      (R)DBMS    -----     Database
sqlite3                     SQLite               FILE
            connect
        cursor(SQL, Params)
        <-----------------> Business Logic != Dev
=> Sns (포스팅 등록, 수정)

In [284]:
import sqlite3

In [285]:
con = sqlite3.connect('sns.db')
cur = con.cursor()

In [286]:
# Table 3개
# post          Hashtag         PostHashtag(N:M)
# PK, content   PK, Tag, Cnt    P.PK, H.PK
cur.executescript('''
    drop table if exists post;
    create table post(
        pno integer primary key,
        content text not null
    );
    
    drop table if exists hashtag;
    create table hashtag(
        hno integer primary key,
        name text not null,
        cnt integer not null default 0
    );

drop table if exists ph;
    create table ph(
        pno integer not null,
        hno integer not null
    );
''')

In [287]:
# 새로운 게시물 등록 (내용, 해시태그들)
# 해시태그; (1) 태그풀에 존재하는 애들만, (2)존재하지 않으면 새롭게 추가
def findTag(tag):
    cur.execute('select hno from hashtag where name=?', (tag,))
    result = cur.fetchone()
    if not result:
        return 0
    else: # none이 아닐 때, fetchone이니깐 tuple(열, 열, ...)
        return result[0]

def addTag(tag):
    cur.execute('insert into hashtag(name) values(?)', (tag,))
    con.commit()
    return cur.lastrowid

findTag('aadsfds'), addTag('태그1')

(0, 1)

In [288]:
findTag('태그1')

1

In [289]:
def plusCount(hno):
    cur.execute('update hashtag set cnt = cnt + 1 where hno=?', (hno,))
    con.commit()

def minusCount(hno):
    cur.execute('update hashtag set cnt = cnt - 1 where hno=?', (hno,))
    con.commit()

In [290]:
plusCount(findTag('태그1')), plusCount(findTag('태그1')), plusCount(findTag('태그1'))

(None, None, None)

In [291]:
plusCount(findTag('태그2')), plusCount(findTag('태그3'))

(None, None)

In [292]:
def addPosting(content, opt=True, *tags):
    cur.execute('insert into post(content) values(?)', (content,))
    con.commit()
    pno = cur.lastrowid
    # if cur.lastrowid < 1:
    #     return None
    
    for tag in tags:
        # (1) 태그풀에 존재하는 애들만
        r = findTag(tag)
        # (2) 태그풀에 존재하지 않으면 추가
        if r == 0 and opt:
            r = addTag(tag) # 새로운 태그가 들어가면, return lastrowid(마지막에 들어간 행번호)
        if r > 0:
            plusCount(r) # 사용된 태그에 +1
            cur.execute('insert into ph(pno, hno) values(?,?)', (pno, r))
            con.commit()
    
    return pno

In [293]:
addPosting('내용2', False, '태그1', '태그2', '태그3')

1

In [294]:
cur.execute('select * from post')
cur.fetchall()

[(1, '내용2')]

In [53]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 4)]

In [54]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(1, 1)]

In [55]:
addPosting('내용3', True, '태그1', '태그2', '태그3')

2

In [56]:
# 게시글 수정(pno, *태그들)
# 태그들 => 기존 태그들과 비교해서 같으면 냅두고 다르면 수정(추가/삭제)
def findTagsByID(pno):
    result = list()
    cur.execute('select hno from ph where pno=?', (pno,))
    for row in cur.fetchall():
        result.append(row[0])
    return result

def modifyPost(pno, content, *tags):
    cur.execute('update post set content=? where pno=?', (content, pno))
    con.commit()

    tlist = findTagsByID(pno) # 원본에 붙어있던 태드글
    hlist = list() # 새롭게 수정된 태그들
    for tag in tags:
        hno = findTag(tag)
        if hno > 0:
            hno = addTag(tag)
        hlist.append(hno)
    removelist = list(set(tlist) - set(hlist))
    pluslist = list(set(hlist) - set(tlist))
    for r in removelist:
        minusCount(r)
        cur.execute('delete from ph where pno=? and pno=?', (pno, r))
    for r in pluslist:
        plusCount(r)
        cur.execute('insert into ph values(?,?)', (pno, r))
    con.commit()

In [57]:
modifyPost(1, '수정된 내용1', '태그1', '태그2') # 태그1 존치, 태그2 추가

In [58]:
modifyPost(2, '수정된 내용2', '태그3') # 태그1이 삭제, 태그3이 추가

In [59]:
modifyPost(2, '수정된 내용3', '태그1', '태그2', '태그3', '태그4') # 태그4가 추가

In [60]:
cur.execute('select * from hashtag')
cur.fetchall()

[(1, '태그1', 3),
 (2, '태그2', 0),
 (3, '태그3', 0),
 (4, '태그1', 1),
 (5, '태그2', 1),
 (6, '태그3', 0),
 (7, '태그1', 1),
 (8, '태그2', 1),
 (9, '태그3', 1)]

In [61]:
#절대 사용하지 말것
cur.execute('delete from post')
cur.execute('delete from hashtag')
cur.execute('delete from ph')
con.commit()

In [62]:
# lastrowid => pk
def addTag(tag):
    if findTag(tag) == 0:
        cur.execute('insert into hashtag(name) values(?)', (tag,))
        con.commit()
    return findTag(tag)

In [63]:
addPosting('내용1', True, '태그1')
addPosting('내용2', False, '태그1', '태그2')
addPosting('내용3', True, '태그2', '태그3')

3

In [64]:
cur.execute('select * from hashtag')
cur.fetchall()

[(1, '태그1', 2), (2, '태그2', 1), (3, '태그3', 1)]

In [65]:
cur.execute('select * from ph')
cur.fetchall()

[(1, 1), (2, 1), (3, 2), (3, 3)]

In [66]:
modifyPost(1, '내용1', '태그1', '태그3') # 태그3 추가
modifyPost(2, '내용2', '태그2') # 태그1 삭제
modifyPost(3, '내용3') # 태그 삭제

In [67]:
cur.execute('select * from hashtag')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 1)]

In [68]:
cur.execute('select * from ph')
cur.fetchall()

[(1, 1), (2, 1), (1, 3), (2, 2)]

In [69]:
cur.execute('''
    select a.content, c.name
    from post as a
    inner join ph as b on b.pno = a.pno
    inner join hashtag as c on b.hno=c.hno
''')
cur.fetchall()

[('내용1', '태그1'), ('내용2', '태그1'), ('내용1', '태그3'), ('내용2', '태그2')]

In [70]:
def viewPost(pno):
    cur.execute('''
        select name from hashtag
        inner join ph on ph.hno=hashtag.hno and ph.pno=?
    ''', (pno,))
    tags = list()
    for row in cur.fetchall():
        tags.append(row[0])
    cur.execute('select content from post where pno=?', (pno,))
    content = cur.fetchone()[0]
    return content, ', '.join(tags)
viewPost(3)

('내용3', '')

In [71]:
def search(name, asc = True):
    order = 'asc' if asc else 'desc'
    cur.execute('select name from hashtag where name like ? order by cnt '+order, ('%'+name+'%',))
    result = list()
    for row in cur.fetchall():
        result.append(row[0])
    return result
search('태그')

['태그1', '태그2', '태그3']

In [72]:
cur.execute('select * from hashtag')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 1)]

In [73]:
def searchPostByTag(tag):
    hno = findTag(search(tag)[0])
    cur.execute('select distinct(content) from post inner join ph on ph.pno=post.pno and ph.hno=?', (hno,))
    print(cur.fetchall())
searchPostByTag('태그1')

[('내용1',), ('내용2',)]


In [74]:
con.close()

In [75]:
!dir

zsh:1: command not found: dir


In [79]:
con = sqlite3.connect('test.db')
con.close()

In [80]:
!ls

ORM.pdf           coffee.db-journal db3d.ipynb        sns.db
coffee.db         db2d.ipynb        est.db            test.db


In [87]:
from sqlalchemy.engine import create_engine

In [91]:
engine = create_engine('sqlite:///alchemy.db', echo=True)

In [92]:
con = engine.connect()

In [95]:
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey

In [96]:
meta = MetaData()

In [100]:
pip install Integer

  Preparing metadata (setup.py) ... done
  Created wheel for Integer: filename=integer-1.0.3-py3-none-any.whl size=16527 sha256=8bee081b4a2b7ef68b0fff1bbaee7d32b1eb7a28cf89d7d8a626120079890531
  Stored in directory: /Users/junhi/Library/Caches/pip/wheels/06/82/bd/1be5ebf6a559522c3cbe0f92bedf7554fe52b572b9deeefab3
Successfully built Integer
Note: you may need to restart the kernel to use updated packages.


In [102]:
from sqlalchemy.types import Integer, Text

In [103]:
meta.tables

FacadeDict({})

In [104]:
Table('user', meta, 
Column('pk', Integer, primary_key=True),
Column('name', Text, nullable=False))

Table('user', MetaData(), Column('pk', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', Text(), table=<user>, nullable=False), schema=None)

In [105]:
userObj = meta.tables['user']

In [106]:
meta.create_all(engine)

2025-03-06 11:26:29,592 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:26:29,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-03-06 11:26:29,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:26:29,594 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2025-03-06 11:26:29,594 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:26:29,595 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	pk INTEGER NOT NULL, 
	name TEXT NOT NULL, 
	PRIMARY KEY (pk)
)


2025-03-06 11:26:29,595 INFO sqlalchemy.engine.Engine [no key 0.00021s] ()
2025-03-06 11:26:29,596 INFO sqlalchemy.engine.Engine COMMIT


In [107]:
print(userObj.select())
result = con.execute(userObj.select())

SELECT "user".pk, "user".name 
FROM "user"
2025-03-06 11:26:35,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:26:35,274 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:26:35,275 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ()


In [108]:
result.fetchall()

[]

In [109]:
meta.clear()

In [115]:
con.close()

In [116]:
con = engine.connect()

In [117]:
meta.tables

FacadeDict({})

In [118]:
meta.reflect(engine) # 불러오는것

2025-03-06 11:27:50,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:27:50,336 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:27:50,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:27:50,337 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:27:50,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:27:50,338 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user")
2025-03-06 11:27:50,338 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:27:50,339 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-03-06 11:27:50,339 INFO sqlalchemy.engine.Engine [raw sql] ('user',)
2025-03-06 11:27:50,340 INFO sqlalchemy.engine.Engine 

In [119]:
con.execute(meta.tables['user'].select()).fetchall()

2025-03-06 11:27:51,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:27:51,337 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:27:51,338 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()


[]

In [120]:
Table('info', meta,
Column('pk', Integer, primary_key=True),
Column('name', Text, nullable=False))

Table('info', MetaData(), Column('pk', Integer(), table=<info>, primary_key=True, nullable=False), Column('name', Text(), table=<info>, nullable=False), schema=None)

In [121]:
# meta.create_all(engine) # 내일 오류
#     Client                                    Server
#     Python                                    SQLite

# ORM(Clas)
# Instance->Meta->SQL->engine->Dialect            반영
# Class->Instance->Meta->SQL->engine->Dialect     반영
# Meta(Class, Table, Mapper)

In [122]:
meta.create_all(engine, [meta.tables['info']])

2025-03-06 11:27:52,145 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:27:52,146 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("info")
2025-03-06 11:27:52,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:27:52,146 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("info")
2025-03-06 11:27:52,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:27:52,147 INFO sqlalchemy.engine.Engine 
CREATE TABLE info (
	pk INTEGER NOT NULL, 
	name TEXT NOT NULL, 
	PRIMARY KEY (pk)
)


2025-03-06 11:27:52,147 INFO sqlalchemy.engine.Engine [no key 0.00021s] ()
2025-03-06 11:27:52,148 INFO sqlalchemy.engine.Engine COMMIT


In [123]:
from sqlalchemy.sql import select, insert, join

In [124]:
print(select(), end='\n\n')
print(select(meta.tables['info']), end='\n\n')
print(select(meta.tables['info']).where(meta.tables['info'].c['pk']==1))

SELECT

SELECT info.pk, info.name 
FROM info

SELECT info.pk, info.name 
FROM info 
WHERE info.pk = :pk_1


In [125]:
info1 = select(meta.tables['info']).where(meta.tables['info'].c['pk']==1)

In [126]:
con.execute(info1).fetchall() # 메모리에서 사용함 db랑 상관없음

2025-03-06 11:27:52,803 INFO sqlalchemy.engine.Engine SELECT info.pk, info.name 
FROM info 
WHERE info.pk = ?
2025-03-06 11:27:52,803 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1,)


[]

In [127]:
user = meta.tables['user'] # db랑 상관없음
info = meta.tables['info'] # db랑 상관없음

In [128]:
user.insert().values(pk=1, name='1번사람') # db랑 상관없음
print(user.insert().values(pk=1, name='1번사람').values())
print(user.insert().values(pk=1, name='1번사람').values().compile().params)

INSERT INTO "user" (pk, name) VALUES (:pk, :name)
{'pk': 1, 'name': '1번사람'}


In [129]:
con.execute(user.insert().values(pk=1,name='1번사람')) # db에 반영됨

2025-03-06 11:27:53,335 INFO sqlalchemy.engine.Engine INSERT INTO user (pk, name) VALUES (?, ?)
2025-03-06 11:27:53,335 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (1, '1번사람')


In [130]:
con.execute(user.select()).fetchall()
con.execute(select(user)).fetchall()

2025-03-06 11:27:53,485 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:27:53,485 INFO sqlalchemy.engine.Engine [cached since 2.149s ago] ()
2025-03-06 11:27:53,486 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:27:53,486 INFO sqlalchemy.engine.Engine [cached since 2.149s ago] ()


[(1, '1번사람')]

Help on function select in module sqlalchemy.sql._selectable_constructors:

select(*entities: '_ColumnsClauseArgument[Any]', **__kw: 'Any') -> 'Select[Any]'
    Construct a new :class:`_expression.Select`.
    
    
    .. versionadded:: 1.4 - The :func:`_sql.select` function now accepts
       column arguments positionally.   The top-level :func:`_sql.select`
       function will automatically use the 1.x or 2.x style API based on
       the incoming arguments; using :func:`_sql.select` from the
       ``sqlalchemy.future`` module will enforce that only the 2.x style
       constructor is used.
    
    Similar functionality is also available via the
    :meth:`_expression.FromClause.select` method on any
    :class:`_expression.FromClause`.
    
    .. seealso::
    
        :ref:`tutorial_selecting_data` - in the :ref:`unified_tutorial`
    
    :param \*entities:
      Entities to SELECT from.  For Core usage, this is typically a series
      of :class:`_expression.ColumnElement` a

In [133]:
meta.clear()

In [134]:
con.close()

2025-03-06 11:41:05,912 INFO sqlalchemy.engine.Engine ROLLBACK


In [135]:
engine

Engine(sqlite:///alchemy.db)

In [137]:
con = engine.connect()

In [138]:
meta.reflect(engine) # 다시 table객체들로 만들어줌

2025-03-06 11:41:50,093 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:41:50,093 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:41:50,094 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:41:50,094 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:41:50,094 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:41:50,095 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("info")
2025-03-06 11:41:50,095 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:41:50,096 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user")
2025-03-06 11:41:50,096 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:41:50,096 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND t

In [141]:
type(meta.tables), meta.tables.keys()

(sqlalchemy.util._collections.FacadeDict, dict_keys(['info', 'user']))

In [142]:
type(meta.tables['user'])

sqlalchemy.sql.schema.Table

In [144]:
meta.tables['user'].c['pk'] # Column 객체

Column('pk', INTEGER(), table=<user>, primary_key=True, nullable=False)

In [145]:
# 코어단에서 이렇게 한다는걸 보여주는 것

In [155]:
a = select().select_from(meta.tables['user']).where(meta.tables['user'].c['pk']==1)
print(select().select_from(meta.tables['user']).where(meta.tables['user'].c['pk']==1).compile().params)

{'pk_1': 1}


In [158]:
print(select(meta.tables['user'].c['pk']))

SELECT "user".pk 
FROM "user"


In [159]:
result = con.execute(select(meta.tables['user'].c['pk']))

2025-03-06 11:48:53,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:48:53,274 INFO sqlalchemy.engine.Engine SELECT user.pk 
FROM user
2025-03-06 11:48:53,274 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()


In [160]:
type(result), result.fetchall()

(sqlalchemy.engine.cursor.CursorResult, [])

In [162]:
user = meta.tables['user']
info = meta.tables['info']

In [167]:
print(user.insert().values(name='1번'))
print(user.insert().values(name='1번').compile().params)
print(insert(user).values(name='2번').compile().params)

INSERT INTO "user" (name) VALUES (:name)
{'name': '1번'}
{'name': '2번'}


In [168]:
con.execute(user.insert().values(name='1번'))

2025-03-06 11:51:53,937 INFO sqlalchemy.engine.Engine INSERT INTO user (name) VALUES (?)
2025-03-06 11:51:53,937 INFO sqlalchemy.engine.Engine [generated in 0.00048s] ('1번',)


In [169]:
con.execute(user.insert().values(name='2번'))

2025-03-06 11:51:57,677 INFO sqlalchemy.engine.Engine INSERT INTO user (name) VALUES (?)
2025-03-06 11:51:57,677 INFO sqlalchemy.engine.Engine [cached since 3.741s ago] ('2번',)


In [171]:
con.commit()

2025-03-06 11:52:11,037 INFO sqlalchemy.engine.Engine COMMIT


In [172]:
con.execute(user.select()).fetchall()

2025-03-06 11:52:28,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:52:28,111 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:52:28,111 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()


[(1, '1번'), (2, '2번')]

In [173]:
con.close()
meta.clear()

2025-03-06 11:52:41,218 INFO sqlalchemy.engine.Engine ROLLBACK


In [174]:
con = engine.connect()
meta.reflect(engine)

2025-03-06 11:53:03,242 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:53:03,243 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:53:03,243 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:53:03,244 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 11:53:03,244 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:53:03,245 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("info")
2025-03-06 11:53:03,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:53:03,246 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user")
2025-03-06 11:53:03,246 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 11:53:03,246 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND t

In [175]:
print(meta.tables['user'].select())

SELECT "user".pk, "user".name 
FROM "user"


In [177]:
con.execute(meta.tables['user'].select()).fetchall()

2025-03-06 11:54:17,007 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 11:54:17,008 INFO sqlalchemy.engine.Engine SELECT user.pk, user.name 
FROM user
2025-03-06 11:54:17,008 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()


[(1, '1번'), (2, '2번')]

In [178]:
info = meta.tables['info']

In [180]:
info.insert().values(name='1번 정보')

In [181]:
meta.clear()
con.close()

2025-03-06 11:59:42,741 INFO sqlalchemy.engine.Engine ROLLBACK


In [183]:
engine = create_engine('sqlite:///coffee.db', echo=True)
con = engine.connect()

In [184]:
meta.reflect(engine)

2025-03-06 12:00:23,737 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 12:00:23,738 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 12:00:23,738 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 12:00:23,739 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-06 12:00:23,739 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 12:00:23,739 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("city")
2025-03-06 12:00:23,740 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 12:00:23,740 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("part")
2025-03-06 12:00:23,740 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 12:00:23,741 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("sells")
2025-03-06 12:00:23,741 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-06 1

In [186]:
meta.tables.keys()

dict_keys(['city', 'part', 'sells', 'supplier'])

In [187]:
city = meta.tables['city']
part = meta.tables['part']
sells = meta.tables['sells']
supplier = meta.tables['supplier']

In [193]:
# 여기까지는 메모리 상에서 객체로만 존재
con.execute(city.select()).fetchall()

2025-03-06 12:02:54,762 INFO sqlalchemy.engine.Engine SELECT city.cno, city.name 
FROM city
2025-03-06 12:02:54,763 INFO sqlalchemy.engine.Engine [cached since 42.16s ago] ()


[(1, '성북구'),
 (2, '강북구'),
 (3, '동대문구'),
 (4, '중구'),
 (5, '중량구'),
 (6, '노원구'),
 (7, '강남구'),
 (8, '서초구')]

In [201]:
print(select(city.c['name'], supplier.c['name']).select_from(city, supplier))

SELECT city.name, supplier.name AS name_1 
FROM city, supplier


In [203]:
print(select(city.c['name'], supplier.c['name'])\
    .select_from(city.join(supplier, supplier.c['cno']==city.c['cno'])))

SELECT city.name, supplier.name AS name_1 
FROM city JOIN supplier ON supplier.cno = city.cno


In [207]:
print(select(city.c['name'], supplier.c['name'])\
    .select_from(city.join(supplier, supplier.c['cno']==city.c['cno'])))

AttributeError: 'Select' object has no attribute 'fetchall'

In [212]:
q = select(city.c['name'], supplier.c['name'], part.c['name'], sells.c['price']).select_from(city.join(supplier, supplier.c['cno']==city.c['cno']).join(sells, sells.c['sno']==supplier.c['sno']).join(part, part.c['pno']==sells.c['pno']))

In [213]:
print(q)

SELECT city.name, supplier.name AS name_1, part.name AS name_2, sells.price 
FROM city JOIN supplier ON supplier.cno = city.cno JOIN sells ON sells.sno = supplier.sno JOIN part ON part.pno = sells.pno


In [214]:
con.execute(q).fetchall()

2025-03-06 12:16:05,430 INFO sqlalchemy.engine.Engine SELECT city.name, supplier.name AS name_1, part.name AS name_2, sells.price 
FROM city JOIN supplier ON supplier.cno = city.cno JOIN sells ON sells.sno = supplier.sno JOIN part ON part.pno = sells.pno
2025-03-06 12:16:05,430 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ()


[('성북구', '1호점', '아메리카노', Decimal('10.0000000000')),
 ('성북구', '2호점', '아메리카노', Decimal('10.0000000000')),
 ('성북구', '3호점', '아메리카노', Decimal('10.0000000000')),
 ('동대문구', '1호점', '아메리카노', Decimal('10.0000000000')),
 ('동대문구', '3호점', '아메리카노', Decimal('10.0000000000')),
 ('중구', '2호점', '아메리카노', Decimal('10.0000000000')),
 ('중량구', '1호점', '아메리카노', Decimal('10.0000000000')),
 ('성북구', '1호점', '카페라테', Decimal('15.0000000000')),
 ('강북구', '1호점', '카페라테', Decimal('15.0000000000')),
 ('동대문구', '2호점', '카페라테', Decimal('15.0000000000')),
 ('중구', '1호점', '카페라테', Decimal('15.0000000000')),
 ('중구', '3호점', '카페라테', Decimal('15.0000000000')),
 ('중량구', '2호점', '카페라테', Decimal('15.0000000000')),
 ('노원구', '1호점', '카페라테', Decimal('15.0000000000')),
 ('성북구', '1호점', '에스프레소', Decimal('8.0000000000')),
 ('강남구', '2호점', '에스프레소', Decimal('8.0000000000')),
 ('강남구', '3호점', '에스프레소', Decimal('8.0000000000')),
 ('서초구', '3호점', '아이스티', Decimal('8.0000000000')),
 ('성북구', '1호점', '아이스티', Decimal('20.0000000000')),
 ('성북구', '2호점', '아이스티', D

In [266]:
con.close()
meta.clear()
engine = create_engine('sqlite:///memory:', echo=True)
con = engine.connect()

2025-03-07 08:55:06,278 INFO sqlalchemy.engine.Engine ROLLBACK


In [272]:
meta.drop_all(engine)

2025-03-07 08:56:32,401 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 08:56:32,402 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("artist")
2025-03-07 08:56:32,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:56:32,402 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("album")
2025-03-07 08:56:32,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:56:32,403 INFO sqlalchemy.engine.Engine 
DROP TABLE album
2025-03-07 08:56:32,403 INFO sqlalchemy.engine.Engine [no key 0.00018s] ()
2025-03-07 08:56:32,404 INFO sqlalchemy.engine.Engine 
DROP TABLE artist
2025-03-07 08:56:32,404 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2025-03-07 08:56:32,405 INFO sqlalchemy.engine.Engine COMMIT


In [273]:
Table('artist', meta,\
    Column('pk', Integer, primary_key=True),\
        Column('name', Text),
        extend_existing=True)
        
Table('album', meta,\
    Column('pk', Integer, primary_key=True),\
    Column('name', Text),\
    extend_existing=True)

Table('album', MetaData(), Column('pk', Integer(), table=<album>, primary_key=True, nullable=False), Column('name', Text(), table=<album>), Column('fk', Integer(), ForeignKey('artist.pk'), table=<album>, nullable=False), schema=None)

In [274]:
meta.tables.keys()

dict_keys(['artist', 'album'])

In [275]:
meta.tables['artist'].c['pk']

Column('pk', Integer(), table=<artist>, primary_key=True, nullable=False)

In [276]:

# 기존 Meta에 등록된 Table객체를 덮어쓸때, extend_existing=True
Table('album', meta,
      Column('pk', Integer, primary_key=True),
      Column('name', Text),
      Column('fk', Integer, ForeignKey(meta.tables['artist'].c['pk']), nullable=False),
                                        # SQL: 'ARTIST.PK'
      extend_existing=True)

Table('album', MetaData(), Column('pk', Integer(), table=<album>, primary_key=True, nullable=False), Column('name', Text(), table=<album>), Column('fk', Integer(), ForeignKey('artist.pk'), table=<album>, nullable=False), schema=None)

In [277]:
Table('album', meta, Column('fk', Integer, ForeignKey(meta.tables['artist'].c['pk']), nullable=False), extend_existing=True)

Table('album', MetaData(), Column('pk', Integer(), table=<album>, primary_key=True, nullable=False), Column('name', Text(), table=<album>), Column('fk', Integer(), ForeignKey('artist.pk'), table=<album>, nullable=False), schema=None)

In [256]:
meta.tables['album'].c['fk']

Column('fk', Integer(), ForeignKey('artist.pk'), table=<album>)

In [278]:
meta.create_all(engine)

2025-03-07 08:57:01,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 08:57:01,157 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("artist")
2025-03-07 08:57:01,157 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:57:01,158 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("artist")
2025-03-07 08:57:01,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:57:01,158 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("album")
2025-03-07 08:57:01,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:57:01,159 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("album")
2025-03-07 08:57:01,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 08:57:01,160 INFO sqlalchemy.engine.Engine 
CREATE TABLE artist (
	pk INTEGER NOT NULL, 
	name TEXT, 
	PRIMARY KEY (pk)
)


2025-03-07 08:57:01,160 INFO sqlalchemy.engine.Engine [no key 0.00034s] ()
2025-03-07 08:57:01,162 INFO sqlalchemy.engine.Engine 
CREATE TABLE album (
	pk INTEGER NOT NULL, 
	name 

In [279]:
artist = meta.tables['artist']
album = meta.tables['album']

In [280]:
con.execute(artist.insert().values(name='가수1'))
con.execute(album.insert().values(name='앨범1', fk = 1))

2025-03-07 08:57:02,867 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 08:57:02,868 INFO sqlalchemy.engine.Engine INSERT INTO artist (name) VALUES (?)
2025-03-07 08:57:02,868 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ('가수1',)
2025-03-07 08:57:02,869 INFO sqlalchemy.engine.Engine INSERT INTO album (name, fk) VALUES (?, ?)
2025-03-07 08:57:02,869 INFO sqlalchemy.engine.Engine [generated in 0.00019s] ('앨범1', 1)


In [281]:
print(select(artist.c['name'], album.c['name']).select_from(artist.join(album, artist.c['pk']==album.c['fk'])))


SELECT artist.name, album.name AS name_1 
FROM artist JOIN album ON artist.pk = album.fk


In [283]:
print(select(artist.c['name'], album.c['name']).select_from(artist.join(album)))

SELECT artist.name, album.name AS name_1 
FROM artist JOIN album ON artist.pk = album.fk
